In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    DDD_Train_Decile4 = pd.read_pickle("DDD_Train_Decile4.pkl")
    DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("DDD_Test_Decile4.pkl")
    y_test = pd.read_pickle("DDD_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(DDD_Train_Decile4, DDD_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM DDD Decile 4')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

2264/2264 [==============================] - 7s 3ms/step - loss: 0.4555 - acc: 0.8017 - val_loss: 0.4409 - val_acc: 0.8025
Epoch 36/75
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4571 - acc: 0.8030 - val_loss: 0.4401 - val_acc: 0.8025
Epoch 37/75
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4543 - acc: 0.8039 - val_loss: 0.4396 - val_acc: 0.8025
Epoch 38/75
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4505 - acc: 0.8052 - val_loss: 0.4386 - val_acc: 0.8007
Epoch 39/75
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4476 - acc: 0.8096 - val_loss: 0.4391 - val_acc: 0.7954
Epoch 40/75
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4535 - acc: 0.8087 - val_loss: 0.4392 - val_acc: 0.7954
Epoch 41/75
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4488 - acc: 0.8096 - val_loss: 0.4378 - val_acc: 0.7954
Epoch 42/75
2264/2264 [==============================] - 7s 3ms/ste

2264/2264 [==============================] - 7s 3ms/step - loss: 0.3972 - acc: 0.8238 - val_loss: 0.3865 - val_acc: 0.8307
Epoch 21/75
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4133 - acc: 0.8176 - val_loss: 0.3814 - val_acc: 0.8360
Epoch 22/75
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4098 - acc: 0.8238 - val_loss: 0.3722 - val_acc: 0.8272
Epoch 23/75
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4080 - acc: 0.8277 - val_loss: 0.3830 - val_acc: 0.8289
Epoch 24/75
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4060 - acc: 0.8242 - val_loss: 0.3784 - val_acc: 0.8342
Epoch 25/75
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3969 - acc: 0.8224 - val_loss: 0.3741 - val_acc: 0.8325
Epoch 26/75
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4009 - acc: 0.8264 - val_loss: 0.3790 - val_acc: 0.8325
Epoch 27/75
2264/2264 [==============================] - 6s 3ms/ste

2264/2264 [==============================] - 6s 3ms/step - loss: 0.4059 - acc: 0.8202 - val_loss: 0.3771 - val_acc: 0.8236
Epoch 54/100
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4058 - acc: 0.8229 - val_loss: 0.3768 - val_acc: 0.8236
Epoch 55/100
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3938 - acc: 0.8255 - val_loss: 0.3772 - val_acc: 0.8236
Epoch 56/100
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4049 - acc: 0.8238 - val_loss: 0.3769 - val_acc: 0.8219
Epoch 57/100
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4044 - acc: 0.8255 - val_loss: 0.3770 - val_acc: 0.8236
Epoch 58/100
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3982 - acc: 0.8242 - val_loss: 0.3764 - val_acc: 0.8254
Epoch 88/100
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3989 - acc: 0.8277 - val_loss: 0.3764 - val_acc: 0.8254
Epoch 89/100
2264/2264 [==============================] - 7s 

2264/2264 [==============================] - 6s 3ms/step - loss: 0.4099 - acc: 0.8207 - val_loss: 0.3871 - val_acc: 0.8254
Epoch 43/75
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3967 - acc: 0.8229 - val_loss: 0.3824 - val_acc: 0.8342
Epoch 44/75
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3987 - acc: 0.8269 - val_loss: 0.3943 - val_acc: 0.8254
Epoch 45/75
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4104 - acc: 0.8185 - val_loss: 0.3882 - val_acc: 0.8289
Epoch 46/75
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4067 - acc: 0.8202 - val_loss: 0.3880 - val_acc: 0.8219

Epoch 00046: ReduceLROnPlateau reducing learning rate to 5.0000002374872565e-05.
Epoch 47/75
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4082 - acc: 0.8118 - val_loss: 0.3852 - val_acc: 0.8272
Epoch 48/75
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4061 - acc: 0.8255 - val_loss: 0.3851 - v

Epoch 26/125
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4085 - acc: 0.8216 - val_loss: 0.3855 - val_acc: 0.8131
Epoch 27/125
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4086 - acc: 0.8163 - val_loss: 0.3889 - val_acc: 0.8219
Epoch 28/125
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4088 - acc: 0.8198 - val_loss: 0.3940 - val_acc: 0.8148
Epoch 29/125
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4051 - acc: 0.8180 - val_loss: 0.3808 - val_acc: 0.8254
Epoch 30/125
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4093 - acc: 0.8216 - val_loss: 0.3846 - val_acc: 0.8236
Epoch 31/125
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3998 - acc: 0.8251 - val_loss: 0.3809 - val_acc: 0.8254
Epoch 32/125
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4058 - acc: 0.8216 - val_loss: 0.3793 - val_acc: 0.8289
Epoch 33/125
2264/2264 [========================

2264/2264 [==============================] - 6s 3ms/step - loss: 0.3880 - acc: 0.8282 - val_loss: 0.3738 - val_acc: 0.8377
Epoch 85/125
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3851 - acc: 0.8246 - val_loss: 0.3738 - val_acc: 0.8377
Epoch 86/125
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3966 - acc: 0.8273 - val_loss: 0.3738 - val_acc: 0.8377
Epoch 87/125
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3970 - acc: 0.8220 - val_loss: 0.3738 - val_acc: 0.8377
Epoch 88/125
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3915 - acc: 0.8202 - val_loss: 0.3738 - val_acc: 0.8377
Epoch 89/125
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3921 - acc: 0.8273 - val_loss: 0.3737 - val_acc: 0.8377
Epoch 90/125
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3879 - acc: 0.8282 - val_loss: 0.3737 - val_acc: 0.8377
Epoch 91/125
2264/2264 [==============================] - 6s 

Epoch 8/50
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4429 - acc: 0.8065 - val_loss: 0.4480 - val_acc: 0.7901
Epoch 9/50
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4374 - acc: 0.8083 - val_loss: 0.4239 - val_acc: 0.7989
Epoch 10/50
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4373 - acc: 0.8065 - val_loss: 0.4265 - val_acc: 0.8007
Epoch 11/50
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4262 - acc: 0.8132 - val_loss: 0.4146 - val_acc: 0.8078
Epoch 12/50
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4303 - acc: 0.8140 - val_loss: 0.4107 - val_acc: 0.8042
Epoch 13/50
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4202 - acc: 0.8198 - val_loss: 0.4132 - val_acc: 0.8113
Epoch 14/50
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4184 - acc: 0.8185 - val_loss: 0.4102 - val_acc: 0.8078
Epoch 15/50
2264/2264 [==============================] - 

2264/2264 [==============================] - 6s 3ms/step - loss: 0.4935 - acc: 0.7743 - val_loss: 0.4654 - val_acc: 0.8025
Epoch 3/100
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4704 - acc: 0.7968 - val_loss: 0.4458 - val_acc: 0.8095
Epoch 4/100
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4637 - acc: 0.7999 - val_loss: 0.4460 - val_acc: 0.8078
Epoch 5/100
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4539 - acc: 0.8021 - val_loss: 0.4357 - val_acc: 0.8095
Epoch 6/100
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4501 - acc: 0.8030 - val_loss: 0.4423 - val_acc: 0.7989
Epoch 7/100
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4469 - acc: 0.8021 - val_loss: 0.4370 - val_acc: 0.7919
Epoch 8/100
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4410 - acc: 0.8034 - val_loss: 0.4353 - val_acc: 0.7989
Epoch 9/100
2264/2264 [==============================] - 7s 3ms/ste

Epoch 61/100
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3888 - acc: 0.8286 - val_loss: 0.3784 - val_acc: 0.8325
Epoch 62/100
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3946 - acc: 0.8233 - val_loss: 0.3781 - val_acc: 0.8325
Epoch 63/100
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3969 - acc: 0.8260 - val_loss: 0.3780 - val_acc: 0.8325
Epoch 64/100
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3923 - acc: 0.8273 - val_loss: 0.3780 - val_acc: 0.8307
Epoch 65/100
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3831 - acc: 0.8299 - val_loss: 0.3779 - val_acc: 0.8307
Epoch 66/100
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3980 - acc: 0.8216 - val_loss: 0.3778 - val_acc: 0.8307
Epoch 67/100
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4039 - acc: 0.8277 - val_loss: 0.3780 - val_acc: 0.8307
Epoch 68/100
2264/2264 [========================

In [ ]:
0.86434